# Description

## Introduction

    The main goal of the **Description** module is to introduce means to enrich the PropaPhen Ontology in order to acquire a domain-specific ontology for a specific phenomenon

In [1]:
%load_ext autoreload
%autoreload 2

## Libraries

### Installation

In [2]:
#!pip install owlready2
#!pip install tqdm
#!pip install pkt_kg

Defaulting to user installation because normal site-packages is not writeable
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


### Standard

In [3]:
from owlready2 import *
import pandas as pd
from tqdm import tqdm

### Custom

## Globals

In [4]:
path_propaphen = "../PropaPhen/PropaPhen.owl"
path_to_mrrel = "data/MRREL.RRF"
path_to_srdef = "data/SRDEF"
path_to_worldkg_ontology = "data/WorldKG_Ontolgy.owl"
path_to_worldkg_nodes = "data/worldkg_nodes.csv"
path_to_worldkg_edges = "data/worldkg_edges.csv"

In [73]:
path_save_umlsonto = "data/saved/umlsonto.owl"
path_save_worldkg = "data/saved/worldkg.owl"

## Ontologies

### PropaPhen

In [6]:
propaphen = get_ontology("file://" + path_propaphen)

In [8]:
propaphen = propaphen.load()   

In [9]:
propaphen_classes = list(propaphen.classes())
propaphen_dict_classes = dict(zip([x.__name__ for x in propaphen_classes],propaphen_classes))

# UMLS


---

UMLS has as main concepts ([ref](https://www.nlm.nih.gov/research/umls/new_users/online_learning/Meta_005.html)):
 - Concept Unique Identifier (CUI):
 - Lexical (term) Unique Identifiers (LUI)
 - String Unique Identifiers (SUI)
 - Atom Unique Identifier (AUI):

Other than those concepts, UMLS also counts with a Semantic Network that consists of (1) a set of broad subject categories, or **Semantic Types**, that provide a consistent categorization of all concepts represented in the UMLS Metathesaurus, and (2) a set of useful and important relationships, or Semantic Relations, that exist between Semantic Types.

---
This subsection describes the aforementioned concepts and retrieves all properties found in the UMLS 2023AB repository

In [69]:
umlsonto = get_ontology("https://w3id.org/def/umls")
# Adding concepts/classes
with umlsonto:
    class UMLSEntity(Thing):
        pass
    class CUI(UMLSEntity):
        pass
    class LUI(UMLSEntity):
        pass
    class SUI(UMLSEntity):
        pass
    class AUI(UMLSEntity):
        pass
    class SemanticType(UMLSEntity):
        pass
    AllDisjoint([CUI, LUI, SUI, AUI, SemanticType])
    UMLSEntity.equivalent_to = [CUI | LUI | SUI | AUI | SemanticType]

#### Adding CUI/AUI relationships

In [12]:
aui_rel = set()
cui_rel = set()
aui_cui_rel = set()
cui_aui_rel = set()

In [13]:
with open(path_to_mrrel, mode= 'r', encoding= 'utf-8') as mrrel: 
    for line in tqdm(mrrel):
        line = line.split("|")
        start_node= line[4]
        end_node= line[0]
        # aui and aui
        if line[6]== 'AUI' and line[2]== 'AUI': # style 2
            aui_rel.add(line[3])  
        elif line[6]== 'AUI': # style 2
            cui_aui_rel.add(line[3])
        if line[2]== 'AUI': # syle 1
            aui_cui_rel.add(line[3])
        else:
            cui_rel.add(line[3])

55685992it [01:03, 882529.73it/s]


In [14]:
cui_domain=cui_rel | cui_aui_rel
aui_domain= aui_cui_rel | aui_rel
cui_range = cui_rel | aui_cui_rel
aui_range = aui_rel | cui_aui_rel

In [15]:
all_rel = list((cui_rel | cui_aui_rel | aui_cui_rel | aui_rel) -  set([str(x) for x in range(10)]))

In [16]:
with umlsonto:
    for rel in all_rel:
        opdomain = []
        oprange = []
        if rel in cui_domain:
            opdomain.append(CUI)
        if rel in aui_domain:
            opdomain.append(AUI)
        if rel in cui_range:
            oprange.append(CUI)
        if rel in aui_range:
            oprange.append(AUI)
        addRelation = type(rel,(ObjectProperty,),  {
            'domain' : opdomain,
            'range' : oprange
        })

#### Adding Semantic Network relationships

In [17]:
semanticrels = pd.read_csv(path_to_srdef,sep='|', header=None, usecols=[0,1])

In [18]:
semanticrelslist = semanticrels[semanticrels[0]=='RL'][1].tolist()

In [19]:
# For each relation
with umlsonto:
    for rel in semanticrelslist:
        addRelation = type(rel,(ObjectProperty,),  {
            'domain' : [SemanticType],
            'range' : [SemanticType]
        })

#### isa type

In [20]:
with umlsonto:
    addRelation = type("isa",(ObjectProperty,),  {
            'domain' : [SemanticType],
            'range' : [SemanticType]
        })

#### CUI and Semantic Network

In [21]:
# For each relation
with umlsonto:
    addRelation = type("STY",(ObjectProperty,),  {
        'domain' : [CUI],
        'range' : [SemanticType]
    })

In [61]:
umlsonto.save(file=path_save_umlsonto,format="rdfxml")

### World-KG

In [23]:
worldkg = get_ontology("file://" + path_to_worldkg_ontology)

In [25]:
worldkg = worldkg.load()

In [26]:
worldkg_dict_classes = dict(zip([x.__name__ for x in 
                                worldkg.classes()],
                               worldkg.classes()))

In [27]:
kgnodes = pd.read_csv(path_to_worldkg_nodes)

/tmp/ipykernel_68447/1074395577.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265

In [28]:
dict_label = dict(zip(kgnodes.loc[:,"id:ID"],kgnodes.loc[:,":LABEL"]))

In [29]:
dict_str_to_class = {}
for value in set(dict_label.values()):
    dict_str_to_class[value] = worldkg_dict_classes[value.split(":")[-1]]

In [30]:
kgedges = pd.read_csv(path_to_worldkg_edges)

In [31]:
wkgdomain = {}
wkgrange = {}
for worldtype in set(dict_label.values()):
    wkgdomain[worldtype] = set()
    wkgrange[worldtype] = set()

In [32]:
all_rel = set()
with open(path_to_worldkg_edges, mode= 'r', encoding= 'utf-8') as mrrel: 
    count = 0
    for line in tqdm(mrrel):
        if count == 0:
            count += 1
            continue
        line = line.split(",")
        property_name = line[2].replace("\n","")
        all_rel.add(property_name)
        domain_obj = dict_label[line[0]]
        range_obj = dict_label[line[1]]
        wkgdomain[domain_obj].add(property_name)
        wkgrange[range_obj].add(property_name)

2228040it [00:03, 737379.83it/s]


In [33]:
# For each relation
with worldkg:
    for rel in all_rel:
        domainlist = []
        rangelist = []
        for key in wkgdomain.keys():
            if rel in wkgdomain[key]:
                domainlist.append(dict_str_to_class[key])
        for key in wkgrange.keys():
            if rel in wkgrange[key]:
                rangelist.append(dict_str_to_class[key])
        addRelation = type(rel,(ObjectProperty,),  {
            'domain' : domainlist,
            'range' : rangelist
        })

In [34]:
worldkg.save(file=path_save_worldkg,format="rdfxml")

## Semi-Automatic Alignment

In [35]:
propaphen.imported_ontologies.append(umlsonto)
propaphen.imported_ontologies.append(worldkg)

In [85]:
# Add wordkg as spatial object
for x in list(worldkg.classes()):
    if Thing in x.is_a:
        x.is_a.remove(Thing)
    x.is_a.append(propaphen_dict_classes['System'])

In [87]:
# Adding umls entities to gufo
umlsonto.search(iri='*UMLSEntity')[0].is_a = [propaphen_dict_classes['Entity']]

In [88]:
worldkg.save(file=path_save_worldkg,format="rdfxml")
umlsonto.save(file=path_save_umlsonto,format="rdfxml")
propaphen.save(file="data/saved/propaphenplus.owl",format="rdfxml")